# Get DHV XC Data

In [1]:
import requests
import urllib
import pandas as pd

PAGE_SIZE = 500

places = {
    'Metzingen':11185,
    'Rammelsberg': 9427,
    'Estorf': 11001,
    'Leese': 10746,
    'Porta': 9712,
    'Lüdingen':9759,
    'Brunsberg': 9844,
    'Kella': 9521,
    'Börry': 9403
}

results = []
for place in ['Metzingen','Estorf','Leese','Lüdingen']:
    limit = PAGE_SIZE
    for year in [2017,2018,2019,2020,2021,2022,2023,2024]:
        query = {"navpars":{"start":0,"limit":PAGE_SIZE,"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}}
        # for some reason, " is replaced by ' and spaces are added which is replaced by +
        decoded_url = f"https://de.dhv-xc.de/api/fli/flights?y={year}&fkcat%5B%5D=1&fkto%5B%5D={places[place]}&{urllib.parse.urlencode(query,quote_via=urllib.parse.quote_plus).replace('%27', '%22').replace('+', '')}"
        #print(decoded_url)
        r = requests.get(decoded_url)
        if r.status_code==200:
            response = r.json()
            df0 = pd.DataFrame(response['data'])
            #print(df.columns.values)
            results.extend(df0[['FlightDate', 'TakeoffWaypointName' , 'Glider' ,'FlightDuration']].values)


df_results = pd.DataFrame(results,columns=['FlightDate','Takeoff', 'Glider' ,'Duration'])
df_results = df_results.astype(dtype= {"Duration":"int64"})
print(len(df_results))
df_results.sort_values(by='Duration').tail()

5681


,FlightDate,Takeoff,Glider,Duration
4198,2019-06-09,Lüdingen,Zeno - S [LTF D],23778
3565,2017-05-20,Lüdingen,Base - L [LTF B],23816
3496,2017-06-19,Lüdingen,Mantra M6,24111
4216,2019-05-18,Lüdingen,MantraM7 - MS [LTF D],26553
4240,2019-05-14,Lüdingen,Cayenne5 - XS [LTF C],27315


In [2]:
from datetime import datetime

dhv_aggr = df_results.groupby('FlightDate')['Duration'].agg(['count','sum']).reset_index()

dhv_aggr = dhv_aggr.merge(df_results[df_results['Takeoff']=='Metzingen'].groupby('FlightDate')['Duration'].agg('count').reset_index(), on='FlightDate', how='left').fillna(0)

dhv_aggr.rename(columns={'FlightDate':'time','Duration':'count_M'},inplace=True)

print(len(dhv_aggr))

dhv_aggr.head()

428


,time,count,sum,count_M
0,2017-01-06,4,2470,0.0
1,2017-01-28,16,6069,8.0
2,2017-02-15,14,6272,0.0
3,2017-03-11,12,7001,2.0
4,2017-03-16,13,8906,0.0


# Get OpenMeteo Data

In [3]:
import requests
import pandas as pd
from datetime import timedelta

LOCATIONS = {
    'Metzingen' : {'lat':52.67057, 'long':10.37136},
    'Hannover Station 2014':  {'lat':52.4644, 'long':9.6779}
    }


DAILY = [
  'sunshine_duration',
  'rain_sum',
  'wind_speed_10m_max',
  'wind_gusts_10m_max',
  'wind_direction_10m_dominant',
  'temperature_2m_mean',
  'precipitation_hours'
]


dates = [
      {'StartDate': '2017-01-01', 'EndDate': '2018-12-31'},
      {'StartDate': '2019-01-01', 'EndDate': '2020-12-31'},
      {'StartDate': '2021-01-01', 'EndDate': '2022-12-31'},
      {'StartDate': '2023-01-01', 'EndDate': '2024-12-14'}
]

loc = 'Metzingen'

dfs = []
for date in dates:
  url = f'https://archive-api.open-meteo.com/v1/archive?latitude={LOCATIONS[loc]["lat"]}&longitude={LOCATIONS[loc]["long"]}&start_date={date["StartDate"]}&end_date={date["EndDate"]}&daily={",".join(DAILY)}&timezone=GMT'
  #print(url)
  res = requests.get(url)
  if res.status_code == 200:
    data = res.json()
    dfs.append(pd.DataFrame(data['daily'], columns= data['daily_units']))


df = pd.concat(dfs)
df['Weekday'] = df.apply(lambda row:datetime.strftime(datetime.strptime(row['time'],'%Y-%m-%d'),'%a'), axis=1)
df['prev_time'] = df.apply(lambda row:datetime.strftime(datetime.strptime(row['time'],'%Y-%m-%d')-timedelta(days=1),'%Y-%m-%d'), axis=1)

df.rename(columns={
    'sunshine_duration':'sun',
    'rain_sum':'rain',
    'wind_speed_10m_max':'wind',
    'wind_gusts_10m_max':'gust',
    'wind_direction_10m_dominant':'dir',
    'temperature_2m_mean': 'temp',
    'precipitation_hours': 'rain_dur'
    },inplace=True)


df_prev= df[['time','sun','rain','wind','gust','dir','rain_dur','temp']].copy(deep=True)

df_prev.rename(columns={k:'prev_'+k for k in ['time','sun','rain','wind','gust','dir','rain_dur','temp']} ,inplace=True)

df = df.merge(df_prev, on='prev_time',how='left')

df.head()

,time,sun,rain,wind,gust,dir,temp,rain_dur,Weekday,prev_time,prev_sun,prev_rain,prev_wind,prev_gust,prev_dir,prev_rain_dur,prev_temp
0,2017-01-01,0.00,0.2,17.4,34.2,235,1.2,6.0,Sun,2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-02,14223.38,0.4,19.2,37.1,272,0.9,11.0,Mon,2017-01-01,0.00,0.2,17.4,34.2,235.0,6.0,1.2
2,2017-01-03,0.00,6.9,34.5,68.0,256,2.2,16.0,Tue,2017-01-02,14223.38,0.4,19.2,37.1,272.0,11.0,0.9
3,2017-01-04,12665.65,7.6,33.8,80.3,296,2.4,16.0,Wed,2017-01-03,0.00,6.9,34.5,68.0,256.0,16.0,2.2
4,2017-01-05,22850.84,0.0,18.3,41.0,2,-3.8,1.0,Thu,2017-01-04,12665.65,7.6,33.8,80.3,296.0,16.0,2.4


# Merge flight and weather data

In [6]:
dfm = df.merge(dhv_aggr, on='time', how='left').fillna(0)
filter = (dfm['Weekday'].isin(['Sat','Sun'])) | (dfm['count'] > 0) # ???

dfm=dfm[filter]

# true ratios
filters = [
    ((dfm['time'] > '2022-12-31') & (dfm['Weekday'].isin(['Sat','Sun']))),
    (dfm['time'] > '2022-12-31'),
    (dfm['time'] > '2020-12-31')  & (dfm['Weekday'].isin(['Sat','Sun'])),
    (dfm['time'] > '2020-12-31'),    
    (dfm['time'] > '2018-12-31')  & (dfm['Weekday'].isin(['Sat','Sun'])),
    (dfm['time'] > '2018-12-31'),
    (dfm['Weekday'].isin(['Sat','Sun'])),
    (dfm['time'] > '2016-12-31'),
    ]
for filter in filters:
    print(f"{len(filter)} Metzingen: {len(dfm[filter & (dfm['count_M']>0)])/len(dfm[filter]):.4f}, others: {len(dfm[filter & (dfm['count']>0)])/len(dfm[filter]):.4f}")

dfm.head()

1039 Metzingen: 0.1471, others: 0.2353
1039 Metzingen: 0.1872, others: 0.3362
1039 Metzingen: 0.1211, others: 0.2397
1039 Metzingen: 0.1383, others: 0.3707
1039 Metzingen: 0.1159, others: 0.2432
1039 Metzingen: 0.1367, others: 0.3824
1039 Metzingen: 0.1325, others: 0.2639
1039 Metzingen: 0.1492, others: 0.4119


,time,sun,rain,wind,gust,dir,temp,rain_dur,Weekday,prev_time,prev_sun,prev_rain,prev_wind,prev_gust,prev_dir,prev_rain_dur,prev_temp,count,sum,count_M
0,2017-01-01,0.00,0.2,17.4,34.2,235,1.2,6.0,Sun,2016-12-31,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2017-01-06,22971.76,0.0,11.0,19.8,201,-6.7,0.0,Fri,2017-01-05,22850.84,0.0,18.3,41.0,2.0,1.0,-3.8,4.0,2470.0,0.0
6,2017-01-07,0.00,0.4,19.1,39.2,235,-3.3,9.0,Sat,2017-01-06,22971.76,0.0,11.0,19.8,201.0,0.0,-6.7,0.0,0.0,0.0
7,2017-01-08,0.00,0.1,12.3,22.3,305,0.3,1.0,Sun,2017-01-07,0.00,0.4,19.1,39.2,235.0,9.0,-3.3,0.0,0.0,0.0
13,2017-01-14,20642.83,0.6,22.0,48.6,281,0.5,7.0,Sat,2017-01-13,2830.21,2.5,26.0,51.8,256.0,16.0,0.6,0.0,0.0,0.0


# Training

In [60]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np

features = ['wind','sun','gust','rain']
features = features + ['prev_'+k for  k in features]
#features = ['prev_'+k for  k in features]

train_filter = (dfm['time'] < '2024-01-01')

xtrain = dfm[train_filter][features].to_numpy()

ytrain = [np.log(y) if y > 0 else 0 for y in dfm[train_filter]['count'].to_numpy()]

print("="*5,"Forest","="*100)
regr = RandomForestRegressor(random_state=0,n_estimators=200
                                # ,max_depth=10,max_features=1,criterion='squared_error'
                            )
regr.fit(xtrain,ytrain)

print({k:regr.feature_importances_[j] for j,k in enumerate(features)} )

ypred = regr.predict(xtrain)
print(((ytrain-ypred)**2).mean())
for thr in [0.,0.5,1.,1.5,2.]:
    tp = len([1 for j in range(len(ytrain)) if ytrain[j] > 0 and ypred[j] >= thr])
    fp = len([1 for j in range(len(ytrain)) if ytrain[j] == 0 and ypred[j] >= thr])
    fn = len([1 for j in range(len(ytrain)) if ytrain[j] > 0 and ypred[j] < thr])
    prec, rec = tp/(tp+fp) , tp/(tp+fn)
    f1score = 2.0*prec*rec/(prec+rec)
    print(f'threshold: {thr:.4f}, {prec:.4f}, {rec:.4f}, f1-score: {f1score:.4f}')


scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
for k in [5,10,20,50,100]:
    print("="*5,f"K-nearest neightbor, k={k}","="*0)
    model = KNeighborsRegressor(n_neighbors=k)

    model.fit(xtrain_scaled,ytrain)
    ypred =  model.predict(xtrain_scaled)
    for thr in [0.,0.5,1.,1.5,2.]:
        tp = len([1 for j in range(len(ytrain)) if ytrain[j] > 0 and ypred[j] >= thr])
        fp = len([1 for j in range(len(ytrain)) if ytrain[j] == 0 and ypred[j] >= thr])
        fn = len([1 for j in range(len(ytrain)) if ytrain[j] > 0 and ypred[j] < thr])
        prec, rec = tp/(tp+fp) , tp/(tp+fn)
        f1score = 2.0*prec*rec/(prec+rec)
        print(f'threshold: {thr:.4f}, {prec:.4f}, {rec:.4f}, f1-score: {f1score:.4f}')    



===== Forest ====================================================================================================
{'wind': 0.32859355856529043, 'sun': 0.2552256622980519, 'gust': 0.1280598489150236, 'rain': 0.04892310476619021, 'prev_wind': 0.05034254991836996, 'prev_sun': 0.09644932978454183, 'prev_gust': 0.05766331266536998, 'prev_rain': 0.0347426330871622}
0.11093065756026249
threshold: 0.0000, 0.3980, 1.0000, f1-score: 0.5694
threshold: 0.5000, 0.8750, 0.9918, f1-score: 0.9298
threshold: 1.0000, 0.9914, 0.9373, f1-score: 0.9636
threshold: 1.5000, 1.0000, 0.8420, f1-score: 0.9142
threshold: 2.0000, 1.0000, 0.6349, f1-score: 0.7767
===== K-nearest neightbor, k=5 
threshold: 0.0000, 0.3980, 1.0000, f1-score: 0.5694
threshold: 0.5000, 0.7066, 0.9646, f1-score: 0.8157
threshold: 1.0000, 0.8034, 0.8910, f1-score: 0.8450
threshold: 1.5000, 0.8786, 0.7493, f1-score: 0.8088
threshold: 2.0000, 0.9577, 0.4932, f1-score: 0.6511
===== K-nearest neightbor, k=10 
threshold: 0.0000, 0.3980, 1.0000

In [69]:
from keras import Sequential, layers
import numpy as np

model = Sequential()
model.add(layers.Dense(100, activation='relu', input_shape=((len(features),)) ))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", metrics=["accuracy"])

model.fit(np.array(xtrain_scaled), np.array([0 if yy < 1. else 1 for yy in ytrain]), epochs=100, verbose=0)

ypred = model.predict(xtrain_scaled)
#print(((ytrain-ypred)**2).mean())


C:\Users\Akos Rapp\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [70]:
print("="*5,"NN","="*100)
for thr in [0.,0.25,0.5,0.75,.9]:
    tp = len([1 for j in range(len(ytrain)) if ytrain[j] > 0 and ypred[j] >= thr])
    fp = len([1 for j in range(len(ytrain)) if ytrain[j] == 0 and ypred[j] >= thr])
    fn = len([1 for j in range(len(ytrain)) if ytrain[j] > 0 and ypred[j] < thr])
    #print(f"tp {tp}, fp {fp}, fn {fn}")
    if tp > 0:
        prec, rec = tp/(tp+fp) , tp/(tp+fn)
        f1score = 2.0*prec*rec/(prec+rec)
        print(f'threshold: {thr:.4f}, {prec:.4f}, {rec:.4f}, f1-score: {f1score:.4f}')
    else:
        print(f"threshold: {thr:.4f}, {'---'}, {'---'}, f1-score: {'---'}")

===== NN ====================================================================================================
threshold: 0.0000, 0.3980, 1.0000, f1-score: 0.5694
threshold: 0.2500, 0.7924, 0.9673, f1-score: 0.8712
threshold: 0.5000, 0.8919, 0.8992, f1-score: 0.8955
threshold: 0.7500, 0.9599, 0.7166, f1-score: 0.8206
threshold: 0.9000, 0.9930, 0.3869, f1-score: 0.5569


In [72]:
df_test = dfm[~train_filter].copy(deep=True)
xtest =df_test[features].to_numpy()
ytest =  [np.log(y) if y > 0 else 0 for y in df_test['count'].to_numpy()]
xtest_scaled = scaler.transform(xtest)

ypred = model.predict(xtest_scaled)
ypred_keras = ypred
thr = 0.5
tp = len([1 for j in range(len(ytest)) if ytest[j] > 0 and ypred[j] >= thr])
fp = len([1 for j in range(len(ytest)) if ytest[j] == 0 and ypred[j] >= thr])
fn = len([1 for j in range(len(ytest)) if ytest[j] > 0 and ypred[j] < thr])
print(f"tp {tp}, fp {fp}, fn {fn}")
if tp > 0:
    prec, rec = tp/(tp+fp) , tp/(tp+fn)
    f1score = 2.0*prec*rec/(prec+rec)
    print(f'threshold: {thr:.4f}, precision {prec:.4f}, recall {rec:.4f}, f1-score: {f1score:.4f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
tp 37, fp 7, fn 6
threshold: 0.5000, precision 0.8409, recall 0.8605, f1-score: 0.8506


# Test / evaluate

In [74]:
df_test = dfm[~train_filter].copy(deep=True)
xtest =df_test[features].to_numpy()
ytest =  [np.log(y) if y > 0 else 0 for y in df_test['count'].to_numpy()]

ypred = regr.predict(xtest)
print(((ytest-ypred)**2).mean())

thr = 1.0
tp = len([1 for j in range(len(ytest)) if ytest[j] > 0 and ypred[j] >= thr])
fp = len([1 for j in range(len(ytest)) if ytest[j] == 0 and ypred[j] >= thr])
fn = len([1 for j in range(len(ytest)) if ytest[j] > 0 and ypred[j] < thr])
prec, rec = tp/(tp+fp) , tp/(tp+fn)
f1score = 2.0*prec*rec/(prec+rec)
print(f'threshold: {thr:.4f}, precision {prec:.4f}, recall {rec:.4f}, f1-score: {f1score:.4f}')

df_test['pred'] = df_test.apply(lambda row: np.exp(regr.predict([row[features]])[0]),axis=1)
df_test['keras'] = np.exp(ypred_keras)

print(df_test[(df_test['count'] > 0)|(df_test['pred'] > np.exp(thr))][['time','count_M','count','pred','keras']+features].to_markdown())

0.7007793926437982
threshold: 1.0000, precision 0.8444, recall 0.8837, f1-score: 0.8636
|      | time       |   count_M |   count |     pred |   keras |   wind |     sun |   gust |   rain |   prev_wind |   prev_sun |   prev_gust |   prev_rain |
|-----:|:-----------|----------:|--------:|---------:|--------:|-------:|--------:|-------:|-------:|------------:|-----------:|------------:|------------:|
| 2583 | 2024-01-28 |         0 |       2 |  6.62669 | 2.23884 |   13.3 | 25962.5 |   23.4 |    0   |        20.2 |   25915.8  |        41.4 |         0   |
| 2596 | 2024-02-10 |         0 |       0 |  4.34745 | 1.47191 |   15.8 | 27944.7 |   30.2 |    1.7 |        21.8 |    4730.76 |        42.1 |        12.1 |
| 2632 | 2024-03-17 |         0 |      10 |  8.83029 | 2.62226 |   13.9 | 37021.7 |   25.6 |    0   |        26.7 |   23702.7  |        54   |         1   |
| 2645 | 2024-03-30 |         0 |       0 |  6.48233 | 2.48839 |   12.8 | 39900.9 |   27.4 |    0.1 |        24.3 |   16117.5  